In [1]:
import numpy as np
import cv2
import os
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split

In [2]:
dataset_path = './data/train' 
images = []
labels = []
cloud_types = ['Ac', 'As', 'Cc', 'Ci', 'Cs', 'Ct', 'Cu', 'Ns', 'St', 'Sc']

for cloud_type in cloud_types:
    path = os.path.join(dataset_path, cloud_type)
    for img_name in os.listdir(path):
        img_path = os.path.join(path, img_name)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (128, 128))  # Resize images to a consistent size
        img = img / 255.0  # Normalize pixel values
        images.append(img)

        # Assign labels: 1 for rain-likely clouds (Ns, St, Sc), 0 for others
        label = 1 if cloud_type in ['Ns', 'St', 'Sc'] else 0
        labels.append(label)

images = np.array(images)
labels = np.array(labels)


In [3]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [4]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

c:\Users\surya\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [5]:
history = model.fit(X_train, y_train, epochs=10, validation_split=0.2)

Epoch 1/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 6s 97ms/step - accuracy: 0.6093 - loss: 0.7520 - val_accuracy: 0.7143 - val_loss: 0.5530
Epoch 2/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 4s 91ms/step - accuracy: 0.7244 - loss: 0.5246 - val_accuracy: 0.6815 - val_loss: 0.5626
Epoch 3/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 4s 87ms/step - accuracy: 0.7313 - loss: 0.5064 - val_accuracy: 0.7887 - val_loss: 0.4995
Epoch 4/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 4s 93ms/step - accuracy: 0.7717 - loss: 0.4710 - val_accuracy: 0.7500 - val_loss: 0.5316
Epoch 5/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 4s 87ms/step - accuracy: 0.7888 - loss: 0.4499 - val_accuracy: 0.7351 - val_loss: 0.5420
Epoch 6/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 4s 87ms/step - accuracy: 0.7962 - loss: 0.4241 - val_accuracy: 0.7262 - val_loss: 0.5591
Epoch 7/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 4s 88ms/step - accuracy: 0.7975 - loss: 0.4488 - val_accuracy: 0.7381 - val_loss: 0.6527
Epoch 8/10
42/42 ━━━━━━━━━━━━━━━━━━━━ 4s 84ms/step - accuracy: 0.8125 - loss: 0.4144 - val_accuracy: 0.7619 - v

In [6]:
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print(f'\nTest accuracy: {test_acc}')

14/14 - 0s - 27ms/step - accuracy: 0.7553 - loss: 0.5405

Test accuracy: 0.7553443908691406


In [7]:
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-3,
    decay_steps=10000,
    decay_rate=0.9
)
optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)

In [8]:
predictions = model.predict(X_test)
print("Sample predictions:", predictions[:5])

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
Sample predictions: [[0.3713614 ]
 [0.27276042]
 [0.00300979]
 [0.36225024]
 [0.320739  ]]


In [13]:
test_image_path = './data/test/30.jpg'  # Replace with your test image path
img = cv2.imread(test_image_path)
img = cv2.resize(img, (128, 128))  # Resize to match the input shape
img = img / 255.0  # Normalize pixel values
img = np.expand_dims(img, axis=0)  # Add batch dimension

prediction = model.predict(img)
if prediction[0][0] > 0.5:
    print(f"Rain likely (Confidence: {prediction[0][0]:.2f})")
else:
    print(f"No rain likely (Confidence: {1 - prediction[0][0]:.2f})")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
Rain likely (Confidence: 0.95)


In [14]:
model.save('cloud_rain_predictor.h5')